# Ansible

**Sources**

- [Building an inventory](https://docs.ansible.com/ansible/latest/getting_started/get_started_inventory.html).

## Hello world

To make ansible work we need:

- Control node with ssh client and ansible installed.
- Managed node with ssh server, and python interpreter.
- Provide access from control node to managed node.

The role of the nodes in our examples will be to play docker containers. The following cell creates docker containers and configures access between them.

In [63]:
docker network create ansible_test_network
docker run -itd --rm --name ansible_host --network ansible_test_network python:3.10-alpine
docker run -itd --rm --name ansible_client --network ansible_test_network python:3.10-alpine

# openssh instalation
docker exec ansible_host sh -c "
apk add -q openssh
ssh-keygen -A
mkdir /root/.ssh
/usr/sbin/sshd -D &" > /dev/null

docker exec ansible_client sh -c "
apk add openssh-client &&\
ssh-keygen -t rsa -N \"\" -f /root/.ssh/id_rsa" > /dev/null

export public_key=$(docker exec ansible_client cat /root/.ssh/id_rsa.pub)
docker exec ansible_host sh -c "echo \"$public_key\" >> /root/.ssh/authorized_keys"

9ff67063d7727bd3235d92d786ea091e346c7daeb58bcd0e5e049fa071bf773d
4c2629e8a699e21cfc1e48f6b784ff37ace8e27d1e92bcda894d84e5f9eda6dd
6968562ab6cd2a472716f126bc55d12ffe363a83722573c904c6be70b9286add
Created directory '/root/.ssh'.


**Note** don't forget to stop containres after all.

In [62]:
docker stop ansible_host ansible_client
docker network rm ansible_test_network

ansible_host
ansible_client
ansible_test_network


Now in the control node we need to create an ansible config `inventory.ini` where we just specify managed node.

In [67]:
docker exec ansible_client sh -c  "
cat << EOF > inventory.ini
[myhosts]
ansible_host
EOF"

Finally, try pinging the managed nodes using the command `ansible myhosts -m ping -i inventory.ini`.

In [69]:
docker exec ansible_client ansible myhosts -m ping -i inventory.ini

[WARNING]: Platform linux on host ansible_host is using the discovered Python
interpreter at /usr/local/bin/python3.10, but future installation of another
Python interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.17/reference_appendices/interpreter_discovery.html for more information.
ansible_host | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/local/bin/python3.10"
    },
    "changed": false,
    "ping": "pong"
}


As a result, we got the message `ansible_host | SUCCESS ...`, which tells us that we successfully ping the managed node.